In [1]:
import pandas as pd
from utilities_DL import get_DataSet_and_invalid_dates,get_MultiModel_loss_args_emb_opts,load_init_trainer
from DL_class import MultiModelTrainer, Trainer
from config import get_args
from save_results import build_results_df
from paths import folder_path,file_name,get_save_directory
import time 
import torch
import argparse

from ray_config import get_ray_config
import ray 
from ray import tune 

# ==== GET PARAMETERS ====
# Load config
model_name = 'STGCN'  #'CNN'
args = get_args(model_name)
#args = get_args(model_name = model_name,learn_graph_structure = True)  # MTGNN

# Modification :
args.epochs = 350
args.K_fold = 6   # Means we will use the first fold for the Ray Tuning and the 5 other ones to get the metrics
if torch.cuda.is_available():
    args.device = 'cuda:0'
    args.batch_size = 256
else :
    args.device = 'cpu'
    args.batch_size = 32

args.single_station = True
args.ray = True

args.loss_function_type =   'quantile' #'MSE' 

if args.loss_function_type == 'MSE':
    args.out_dim = 1
    args.alpha = None
    args.type_calendar = 'tuple'
    args.ray_track_pi = False

else:
    args.embedding_dim = 3
    args.calendar_class = 3
    args.position = 'input'
    args.specific_lr = False
    args.type_calendar = 'tuple'
    args.out_dim = 2
    args.alpha = 0.1
    args.ray_track_pi = True

## Search Space : 

In [2]:
config = {"lr": tune.qloguniform(1e-4, 1e-1, 5e-5),
          "weight_decay" : tune.uniform(0.0005, 0.1),
          "momentum" : tune.uniform(0.85, 0.99),
          "dropout" : tune.uniform(0,0.9),
        }

config_embedding = {#'calendar_class' : tune.choice([1,2,3]),
                    'embedding_dim' : tune.choice([2,3,4,5,6]),
                    'multi_embedding' : tune.choice([True,False]),
                    #'TE_transfer' : tune.choice([True,False]),
                    }


config_stgcn = {"Kt" : tune.choice([2,3,4]),
                "stblock_num" : tune.choice([1,2,3,4]),
                "act_fun" : tune.choice(['glu','gtu']),
                "Ks" :  tune.choice([2,3]),
                "graph_conv_type" : tune.choice(['cheb_graph_conv','graph_conv']),
                "gso_type" : tune.choice(['sym_norm_lap', 'rw_norm_lap', 'sym_renorm_adj', 'rw_renorm_adj']),
                "adj_type" : 'dist',
                }

if args.time_embedding:
    config.update(config_embedding)

if args.model_name == 'STGCN':
     config.update(config_stgcn)

## Hyper Parameter Tuning sur le Fold 0

In [3]:
def load_trainer(config,folder_path,file_name,args):

    for key, value in config.items():
        if hasattr(args, key):
            setattr(args, key, value)

    Datasets,DataLoader_list,dic_class2rpz,nb_words_embedding,time_slots_labels,dic_rpz2class = load_init_trainer(folder_path,file_name,args)
    (loss_function,Model_list,Optimizer_list,args_embedding) = get_MultiModel_loss_args_emb_opts(args,nb_words_embedding,dic_class2rpz,n_vertex = len(Datasets[0].columns))
    dataset,dataloader,model,optimizer = Datasets[0],DataLoader_list[0],Model_list[0],Optimizer_list[0]


    trainer = Trainer(dataset,model,dataloader,
                    args,optimizer,loss_function,scheduler = None,
                    args_embedding=args_embedding,
                    save_dir = None,dic_class2rpz=dic_class2rpz)
    return(trainer)

def Train_with_tune(config):
    trainer = load_trainer(config,folder_path,file_name,args)
    result_df = trainer.train_and_valid()



ray_scheduler,ray_search_alg,resources_per_trial,num_gpus,max_concurrent_trials = get_ray_config(args)


if ray.is_initialized:
    ray.shutdown()
    ray.init(num_gpus=num_gpus)

config = {"lr": tune.choice([1e-4])}
analysis = tune.run(
        Train_with_tune,
        config=config,
        num_samples=1,  # Increase num_samples for more random combinations
        resources_per_trial = resources_per_trial,
        max_concurrent_trials = max_concurrent_trials,
        scheduler = ray_scheduler,
        search_alg = ray_search_alg,
    )

2024-05-16 18:42:38,564	ERROR services.py:1207 -- Failed to start the dashboard , return code 0
2024-05-16 18:42:38,566	ERROR services.py:1232 -- Error should be written to 'dashboard.log' or 'dashboard.err'. We are printing the last 20 lines for you. See 'https://docs.ray.io/en/master/ray-observability/ray-logging.html#logging-directory-structure' to find where the log file is.
2024-05-16 18:42:38,666	ERROR services.py:1276 -- 
The last 20 lines of /tmp/ray/session_2024-05-16_18-42-35_978057_9442/logs/dashboard.log (it contains the error message from the dashboard): 
  File "/Users/romainrochas/opt/anaconda3/envs/tuning/lib/python3.11/site-packages/ray/dashboard/head.py", line 327, in run
    self.http_server = await self._configure_http_server(modules)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/romainrochas/opt/anaconda3/envs/tuning/lib/python3.11/site-packages/ray/dashboard/head.py", line 158, in _configure_http_server
    http_server = HttpServ

(Train_with_tune pid=9465) coverage period: 2019-01-01 00:00:00 - 2020-01-01 00:00:00
(Train_with_tune pid=9465) Time-step per hour: 4.0
(Train_with_tune pid=9465) start training
(Train_with_tune pid=9465) Proportion of label with quantile order set to 1: 25.0%


Trial name,_metric
Train_with_tune_4f711_00000,"{'Loss_model': 0.009957676605572776, 'MPIW': 6.385819435119629, 'PICP': 0.5104166666666666}"


(Train_with_tune pid=9465) Proportion of label with quantile order set to 1: 25.0%
(Train_with_tune pid=9465) Proportion of label with quantile order set to 1: 25.0%
(Train_with_tune pid=9465) Proportion of label with quantile order set to 1: 25.0%
(Train_with_tune pid=9465) Proportion of label with quantile order set to 1: 25.0%
(Train_with_tune pid=9465) Proportion of label with quantile order set to 1: 25.0%
(Train_with_tune pid=9465) Proportion of label with quantile order set to 1: 25.0%
(Train_with_tune pid=9465) Proportion of label with quantile order set to 1: 25.0%
(Train_with_tune pid=9465) Proportion of label with quantile order set to 1: 25.0%
(Train_with_tune pid=9465) Proportion of label with quantile order set to 1: 25.0%
(Train_with_tune pid=9465) Proportion of label with quantile order set to 1: 25.0%
(Train_with_tune pid=9465) Proportion of label with quantile order set to 1: 25.0%
(Train_with_tune pid=9465) Proportion of label with quantile order set to 1: 25.0%
(Tra